In [ ]:
import pandas as pd
import datetime as dt
from pymongo import MongoClient

In [ ]:
def merge(reddit, gme,u,p,port):
    '''
    Funzione di merge tra 
    @params:
        reddit:   submission effettuate sul subreddit (dataframe)
        gme:      dati finanziari (dataframe)

    '''
    # Importazione dei dati
    reddit = pd.read_csv(reddit, sep = ",")
    gme = pd.read_csv(gme, sep = ",")
    
    #date in formato datetime
    gme['DATE']=pd.to_datetime(gme['DATE'], format= '%Y-%m-%d %H:%M:%S')
    reddit['Publish Date']=pd.to_datetime(reddit['Publish Date'], format= '%Y-%m-%d %H:%M:%S')  
    
    #Arrotondiamo al quarto d'ora
    reddit['DATE']=reddit['Publish Date'].dt.round('15min')
    reddit['DATE']=pd.to_datetime(reddit['DATE'], format= '%Y-%m-%d %H:%M:%S')
    
    #Merge
    gmereddit=pd.merge(gme,reddit, on='DATE',how='right')
    
    #Salviamo in csv
    gmereddit.to_csv('gmereddit_completo_prepost.csv',index=False)
    
    
    ###To mongo
    #Rename the column 'Total No. of Comments' because it must not contain '.'
    gmereddit = gmereddit.rename(columns={'Total No. of Comments': 'Total No of Comments'})
    
    #file json
    gmereddit.to_json("gmereddit.json",orient="records")

    # Definizione del client mongo da utilizzare
    myclient = MongoClient('localhost', int(port), username = u, password = p)

    db = client["DataMan"] # Connessione al db
    collection = db["gmereddit"] # Connessione alla collection

    gmereddit_collection = db.get_collection("gmereddit")

    with open("gmereddit.json") as f:
        file_data = json.load(f)

    # insert the data into the collection
    gmereddit_collection.insert_many(file_data)


In [ ]:
#Query di prova
gmereddit_collection.find_one()
gmereddit_collection.find_one( {"Score" : 8} )